# Modular Backtester (Documented)
This notebook documents a small backtesting engine designed to run:
- regular standalone models (direct weights)
- micro (asset-level) models
- macro (market-level) models
- an optional portfolio allocator (incl. sector allocation hooks)

All components can run individually, and can also be composed together.


## 0) Setup
Implementation code lives in `src/backtester/` and is imported below.


In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add project root to path so `src/` can be imported
sys.path.append(os.path.abspath('..'))

from src.backtester.data import load_cleaned_assets, align_close_prices
from src.backtester.engine import BacktestConfig, run_backtest
from src.backtester.metrics import compute_performance_stats
from src.backtester.models import (
    SMACrossoverMicroModel,
    RiskOnOffMacroModel,
    TopKLongShortAllocator,
    WeightsFromSignalsModel,
    combine_models_to_weights,
)
from src.backtester.plots import plot_backtest_result, plot_weights_heatmap
from src.backtester.stat_arb import PairTradingModel, compute_pair_diagnostics, plot_pair_diagnostics


## 0.1) Backtest Configuration Options
You can change the two key run-time parameters requested:
- initial capital (starting equity)
- transaction costs (in basis points, applied to turnover)
- **strict_signals**: if True, trades are only executed on signal changes (discrete entry/exit).
- **stop_loss_pct**: if > 0, an intraday safety net sells positions tracking a specific loss % from entry.

These are passed via `BacktestConfig`.


In [ ]:
INITIAL_CAPITAL = 1_000_000
TX_COST_BPS = 5  # 5 bps = 0.05% per unit of turnover

cfg = BacktestConfig(
    initial_equity=INITIAL_CAPITAL,
    transaction_cost_bps=TX_COST_BPS,
    rebalance='D',
    strict_signals=False,
    stop_loss_pct=0.0
)
cfg


## 1) Data Contract
### Required input schema
For each asset we expect a DataFrame with:
- index: `Date` (datetime)
- columns: `Open`, `High`, `Low`, `Close`, `Volume`

### We backtest on Close-to-Close returns
For each asset $i$ and date $t$:
$$r_{t,i} = \frac{P_{t,i}}{P_{t-1,i}} - 1$$

The engine consumes an aligned price matrix $P_t$ and a weights matrix $w_t$.


In [ ]:
assets = load_cleaned_assets(symbols=['Asset_001', 'Asset_002', 'Asset_003', 'Asset_010', 'Asset_011', 'Asset_012'])
close = align_close_prices(assets)
close.tail()


Note: This notebook uses local CSVs under `dataset/cleaned/` via `load_cleaned_assets()`. No live market data is fetched.


## 2) Backtest Engine Math
We use a weights-based, vectorized backtest.

### Portfolio return
We assume weights decided at $t-1$ are held over $(t-1, t]$:
$$r_t^{\text{gross}} = \sum_i w_{t-1,i} r_{t,i}$$

### Turnover and transaction costs
Turnover is the $L^1$ change in weights:
$$T_t = \sum_i |w_{t,i} - w_{t-1,i}|$$

We model costs as linear in turnover with a fixed bps rate:
$$c_t = \frac{\text{bps}}{10{,}000} T_t$$

### Net return and equity curve
$$r_t = r_t^{\text{gross}} - c_t$$
$$E_t = E_0 \prod_{\tau \le t} (1 + r_{\tau})$$

### Risk Management: Intraday Stop Loss
If `stop_loss_pct` is enabled, the engine checks every bar:
$$P_{t, \text{low}} < P_{\text{entry}} \cdot (1 - \text{SL})$$
If triggered, signal is overridden and position is sold at $\max(P_{t, \text{open}}, P_{\text{stop}})$.


## 3) Components: Micro / Macro / Allocator / Regular
We keep components independent:
- Micro model: produces per-asset alpha scores $\alpha_{t,i}$
- Allocator: maps $\alpha$ to weights $w$ (and can enforce sector rules)
- Macro model: produces a portfolio-level risk scale $s_t$ (e.g. 0/1)
- Regular model: can directly output weights (standalone strategy)


### 3.1 Micro model (standalone): SMA crossover -> alpha


In [ ]:
micro = SMACrossoverMicroModel(fast=20, slow=100)
alpha = micro.compute_alpha(assets)
alpha.tail()


### 3.2 Allocator (standalone): Top-K long/short


In [ ]:
allocator = TopKLongShortAllocator(k=2)
w_alloc = allocator.allocate(alpha)
w_alloc.tail()


### 3.3 Macro model (standalone): risk-on / risk-off


In [ ]:
macro = RiskOnOffMacroModel(lookback=200)
risk_scale = macro.compute_risk_scale(assets)
risk_scale.tail()


Macro output is a scalar time series $s_t$. In combination, we scale final weights:
$$w'_{t,i} = s_t \cdot w_{t,i}$$


### 3.4 Regular model (standalone): direct weights


In [ ]:
# Example: equal-weight long-only regular model
regular_w = pd.DataFrame(1.0 / close.shape[1], index=close.index, columns=close.columns)
regular_model = WeightsFromSignalsModel(signals=regular_w)
regular_model.compute_weights().tail()


## 4) Run Backtests
We now run the engine with different inputs to demonstrate:
- each model individually
- micro + allocator
- micro + allocator + macro
- regular model alone


### 4.1 Regular model only


In [ ]:
res_regular = run_backtest(close_prices=close, weights=regular_model.compute_weights(), config=cfg)
stats_regular = compute_performance_stats(equity=res_regular.equity, returns=res_regular.returns)
stats_regular


In [ ]:
plot_backtest_result(result=res_regular, stats=stats_regular, title='Regular model: Equal-weight long-only')


### 4.2 Micro model + allocator


In [ ]:
w_micro = combine_models_to_weights(assets=assets, micro_model=micro, allocator=allocator)
res_micro = run_backtest(close_prices=close, weights=w_micro, config=cfg)
stats_micro = compute_performance_stats(equity=res_micro.equity, returns=res_micro.returns)
stats_micro


In [ ]:
plot_backtest_result(result=res_micro, stats=stats_micro, title='Micro+Allocator: SMA alpha -> TopK long/short')


In [ ]:
plot_weights_heatmap(res_micro.weights.tail(250), title='Weights (last 250 bars)')


### 4.3 Micro + allocator + macro (combined)


In [ ]:
w_combo = combine_models_to_weights(assets=assets, micro_model=micro, allocator=allocator, macro_model=macro)
res_combo = run_backtest(close_prices=close, weights=w_combo, config=cfg)
stats_combo = compute_performance_stats(equity=res_combo.equity, returns=res_combo.returns)
stats_combo


In [ ]:
plot_backtest_result(result=res_combo, stats=stats_combo, title='Micro+Allocator+Macro: risk-scaled strategy')


## 5) Statistical Arbitrage: Pair Trading
We implement a simple pairs strategy using a spread z-score.

### Hedge ratio (OLS)
Given two price series $y_t$ and $x_t$, estimate:
$$y_t \approx a + b x_t$$
where $(a,b)$ minimize $\sum_t (y_t - a - b x_t)^2$.

### Spread and z-score
$$s_t = y_t - (a + b x_t)$$
$$z_t = \frac{s_t - \mu_t}{\sigma_t}$$
with $(\mu_t,\sigma_t)$ computed over a rolling window.

### Trading rule
- enter short-spread when $z_t \ge z_{entry}$
- enter long-spread when $z_t \le -z_{entry}$
- exit when $|z_t| \le z_{exit}$


In [ ]:
y = assets['Asset_001']['Close']
x = assets['Asset_002']['Close']
diag = compute_pair_diagnostics(y=y, x=x, zscore_window=60)
diag


In [ ]:
plot_pair_diagnostics(diag=diag, title='Asset_001 vs Asset_002: Spread + Z-Score')


In [ ]:
pair_model = PairTradingModel(entry_z=2.0, exit_z=0.5, zscore_window=60)
w_pair = pair_model.compute_weights(close_y=y, close_x=x)
pair_prices = pd.concat([y.rename('Y'), x.rename('X')], axis=1).dropna()
res_pair = run_backtest(close_prices=pair_prices, weights=w_pair.reindex(pair_prices.index).fillna(0.0), config=cfg)
stats_pair = compute_performance_stats(equity=res_pair.equity, returns=res_pair.returns)
stats_pair


In [ ]:
plot_backtest_result(result=res_pair, stats=stats_pair, title='Stat Arb: Pair trading (Y=Asset_001, X=Asset_002)')


## 6) Notes on Portfolio / Sector Allocation Input
The allocator interface is where fund allocation / sector constraints belong.
This codebase includes a `SectorAllocationPostProcessor` that can take sector-level
targets (e.g. from a portfolio management model) and apply them to an existing
weights matrix.


In [ ]:
from src.backtester.models import SectorAllocationPostProcessor

# Example sector metadata (placeholder): asset -> sector
sector_map = {
    'Asset_001': 'Tech',
    'Asset_002': 'Tech',
    'Asset_003': 'Energy',
    'Asset_010': 'Energy',
    'Asset_011': 'Finance',
    'Asset_012': 'Finance',
}
sector_targets = {'Tech': 0.4, 'Energy': 0.4, 'Finance': 0.2}
sector_post = SectorAllocationPostProcessor(sector_map=sector_map, sector_gross_targets=sector_targets)

w_with_sector = sector_post.apply(w_micro)
w_with_sector.tail()
